In [12]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os 

WORK_AREA = '/Users/asaf/Workspace/biu/complex-utterance-to-code'
os.chdir(WORK_AREA)

paths = ['./src/', './src/api/v6', './notebooks/src']
for path in paths:
    path = os.path.normcase(path)
    if not any(os.path.normcase(sp) == path for sp in sys.path):
        sys.path.append(path)

In [33]:
from typing import Union, List
import glob
import pandas as pd
import numpy as np
import tqdm
from transformers import GPT2TokenizerFast
import math
import tokenize
from datetime import datetime
import time
import json
import re
from llm.prompts import *

## Evaluation data

In [ ]:
FILE_NAME = 'eval_complex_utterance_to_code_with_intermediate_152_20231112.csv.gz'
BASE_PATH = '/Users/asaf/Workspace/biu/complex-utterance-to-code/build'

def load_eval_data(file_name: str = FILE_NAME, base_path: str = BASE_PATH) -> pd.DataFrame:
    eval_df = pd.read_csv(os.path.join(base_path, file_name))
    eval_df = eval_df.reset_index()  # make sure indexes pair with number of rows
    return eval_df

## Building a prompt

In [34]:
prompt = build_example_prompt(input_value=examples_df['text'][0], input_label="Text", output_value=examples_df['code'][0], output_label="Code")
print(json.dumps(prompt, indent=2))


NameError: name 'examples_df' is not defined

In [18]:
input_data = examples_df.iloc[0]
prompt = build_prompt(strategy="rep2rep", prompt_type='examples', input_data=input_data, examples_df=examples_df, examples_limit=2)
prompt_str = "".join([p['content'] for p in prompt if p['role'] == 'user'])

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
base_prompt_tokens_len = len(tokenizer(prompt_str, max_length=51200, truncation=True)["input_ids"])
print(f"Base prompt tokens length: {base_prompt_tokens_len}")

Base prompt tokens length: 900


In [19]:
examples_df.iloc[0]['lang_rep']

'[ root\n\t[ S\n\t\t[ Command\n\t\t\t[ Action\n\t\t\t\t[ hd\n\t\t\t\t\t[ see ]\n\t\t\t\t]\n\t\t\t\t[ advcl\n\t\t\t\t\t[ mark\n\t\t\t\t\t\t[ if ]\n\t\t\t\t\t]\n\t\t\t\t\t[ Command\n\t\t\t\t\t\t[ Action\n\t\t\t\t\t\t\t[ hd\n\t\t\t\t\t\t\t\t[ find ]\n\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t[ Arg\n\t\t\t\t\t\t\t\t[ obj\n\t\t\t\t\t\t\t\t\t[ nmod:poss\n\t\t\t\t\t\t\t\t\t\t[ my ]\n\t\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t\t\t[ amod\n\t\t\t\t\t\t\t\t\t\t[ first ]\n\t\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t\t\t[ hd\n\t\t\t\t\t\t\t\t\t\t[ reminders ]\n\t\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t[ mark\n\t\t\t\t\t\t\t\t[ that ]\n\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t[ ccomp\n\t\t\t\t\t\t\t\t[ Command\n\t\t\t\t\t\t\t\t\t[ Action\n\t\t\t\t\t\t\t\t\t\t[ Arg\n\t\t\t\t\t\t\t\t\t\t\t[ nsubj\n\t\t\t\t\t\t\t\t\t\t\t\t[ I ]\n\t\t\t\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t\t\t\t[ hd\n\t\t\t\t\t\t\t\t\t\t\t[ have ]\n\t\t\t\t\t\t\t\t\t\t]\n\t\t\t\t\t\t\t\t\t\t[ Arg\n\t\t\t\t\t\t\t\t\t\t\t[ obj\n

In [20]:
re.sub("\s+", " ", examples_df.iloc[0]['lang_rep'])

'[ root [ S [ Command [ Action [ hd [ see ] ] [ advcl [ mark [ if ] ] [ Command [ Action [ hd [ find ] ] [ Arg [ obj [ nmod:poss [ my ] ] [ amod [ first ] ] [ hd [ reminders ] ] ] ] [ mark [ that ] ] [ ccomp [ Command [ Action [ Arg [ nsubj [ I ] ] ] [ hd [ have ] ] [ Arg [ obj [ det [ a ] ] [ hd [ meeting ] ] [ nmod [ case [ at ] ] [ hd [ 3 ] ] [ nmod:tmod [ pm ] ] ] ] ] ] ] [ cc [ and ] ] [ Command_conj [ Condition [ If [ Body [ Command [ Action [ expl [ there ] ] [ hd [ are ] ] ] ] [ cc [ and ] ] [ Command_conj [ Action [ hd [ see ] ] ] ] ] [ mark [ if ] ] [ Test [ Command [ Action [ Arg [ nsubj [ I ] ] ] [ hd [ got ] ] [ Arg [ obj [ det [ a ] ] [ hd [ reminder ] ] [ nmod [ case [ at ] ] [ hd [ mindnight ] ] ] ] ] [ Arg [ obl [ case [ in ] ] [ nummod [ 2 ] ] [ hd [ days ] ] ] ] [ mark [ to ] ] [ advcl [ Command [ Action [ hd [ bring ] ] [ Arg [ obj [ det [ the ] ] [ hd [ keys ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] ] '

In [21]:
input_data = examples_df.iloc[0]
prompt = build_prompt(
    strategy="rep2rep", 
    prompt_type='apispec', 
    input_data=input_data, 
    examples_df=examples_df, 
    examples_limit=2
)
prompt_str = "".join([p['content'] for p in prompt if p['role'] == 'user'])

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
base_prompt_tokens_len = len(tokenizer(prompt_str, max_length=51200, truncation=True)["input_ids"])
print(f"Base prompt tokens length: {base_prompt_tokens_len}")
print(json.dumps(prompt, indent=2))

Base prompt tokens length: 16317
[
  {
    "role": "system",
    "content": "You are a skilled programmer. You will be provided with a variation of a Universal Dependencies (UD) tree and your task is to convert it into a dense variation of an Abstract Syntax Tree (AST) suitable for Python code."
  },
  {
    "role": "user",
    "content": "Below are API specifications for implementing the corresponding dense AST for Python code.\n\n# MAP:\n\nclass Map(Action):\n    \"\"\"\n    The Map class contains all the methods of a virtual assistant agent in the map domain.\n    \"\"\"\n\n    @classmethod\n    def find_on_map(cls, location: Location) -> List[MapEntity]:\n        \"\"\"\n        This class method finds places on the map.\n\n        Parameters\n        ----------\n        location : Location\n            The location to search for\n\n        Returns\n        -------\n        List[MapEntity]\n            A list of places in the form of map entities\n        \"\"\"\n        pass\n\n# 

## TogetherAI Predictions

### gpt-4

#### Examples prompt

In [45]:
# MODEL_NAME = 'gpt-4'
MODEL_NAME = 'gpt-4-1106-preview'

In [399]:
n = 100
id_labels = ['sample_id'] #['test_id', 'sample_id', 'sample_minor_id']
prompt_type = 'examples' # 'examples' or 'apispec'
model_id = 'openai_' + MODEL_NAME.replace('-', '_')
strategy = 'rep2rep'
examples_limit = 18

In [400]:
test_df = load_eval_data()
print(test_df.shape)
test_df.head(3)

(152, 12)


,index,test_id,sample_id,sample_minor_id,text,code,test,imports,lang_rep,lang_rep_raw,code_rep,code_rep_raw
0,0,0,0,NaN,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ root [ hd [ Check ] ] [ obj [ det [ the ] ] ...,[ Module [ product_name = ProductName.resolve_...,[ Module [ Assign [ Name [ product_name ] ] [ ...
1,1,1_a,1,a,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ root [ advcl [ mark [ If ] ] [ nsubj [ it ] ...,[ Module [ date_time = DateTime.resolve_from_t...,[ Module [ Assign [ Name [ date_time ] ] [ Cal...
2,2,1_b,1,b,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ root [ advcl [ mark [ If ] ] [ nsubj [ it ] ...,[ Module [ date_time = DateTime.resolve_from_t...,[ Module [ Assign [ Name [ date_time ] ] [ Cal...


In [401]:
results_file_path = f"./build/results/test-{str(test_df.shape[0])}-{model_id}-{strategy}-n{n}-{prompt_type}-ex{examples_limit}.csv.gz"
print(results_file_path)

./build/results/test-152-openai_gpt_4-text2code-n100-examples-ex18.csv.gz


In [402]:
# loading the results file
def load_test_results(results_file_path: str = results_file_path) -> pd.DataFrame:
    test_results_df = pd.read_csv(results_file_path, compression='gzip') if os.path.exists(results_file_path) else test_df.copy()
    test_results_df.set_index(id_labels, inplace=True)
    test_results_df.sort_index(inplace=True)
    return test_results_df

load_test_results(results_file_path).head(3)

,index,test_id,sample_minor_id,text,code,test,imports,lang_rep,code_rep,output,n
sample_id,,,,,,,,,,,
0,0,0,NaN,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ Module [ product_name = ProductName.resolve_...,"locations = Location.resolve_many_from_text(""W...",0.0
0,0,0,NaN,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ Module [ product_name = ProductName.resolve_...,"product_name = ProductName.resolve_from_text(""...",78.0
0,0,0,NaN,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ Module [ product_name = ProductName.resolve_...,"product_name = ProductName.resolve_from_text(""...",99.0


In [403]:
input_data = examples_df.iloc[0]
prompt = build_prompt(strategy=strategy, prompt_type=prompt_type, input_data=input_data, examples_df=examples_df, examples_limit=examples_limit)
prompt_str = "\n".join([p['content'] for p in prompt if p['role'] == 'user'])

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
base_prompt_tokens_len = len(tokenizer(prompt_str, max_length=51200, truncation=True)["input_ids"])
print(f"Base prompt tokens length: {base_prompt_tokens_len}")

Base prompt tokens length: 606


In [404]:
def run(
    test_results_df,
    openai_disabled = False,
    force = False,
    total_records = test_df.shape[0],
    max_tokens = 500,
    wait_time_in_seconds = 20,
):
    # generate predictions
    responses = []
    print(f"Generating predictions for {total_records} records")
    for i, row  in tqdm.notebook.tqdm(test_df.iterrows(), total=total_records, desc="Processing records"):   
        # check to see if we already have a result for this record
        index = tuple(row[id_label] for id_label in id_labels) if len(id_labels) > 1 else row[id_labels[0]]
        index = [index]
        if (not force) and ('output' in test_results_df.loc[index]) and (not any(pd.isnull(test_results_df.loc[index, 'output']))):
            # if we do, then skip this record
            time.sleep(0.1)
        elif not openai_disabled:
            # run the model, if we don't have a result for this record
            messages = build_prompt(
                strategy=strategy, 
                prompt_type=prompt_type, 
                input_data=row, 
                examples_df=examples_df, 
                examples_limit=examples_limit
            )
            response = openai.ChatCompletion.create(
                model=MODEL_NAME, 
                messages=messages,
                max_tokens=max_tokens,
                n=n
            )
            responses.append(response)
            
            outputs = [x['message']['content'] for x in response['choices']]
            # outputs = [output.replace("Code:\n", "").replace("```python", "").replace("```py", "").replace("```", "").strip() for output in outputs]
            ns = list(np.arange(n))
            
            # duplicate the records
            records_to_duplicate = test_results_df.loc[index] # Fetch the records
            duplicated_records = pd.concat([records_to_duplicate] * (n - 1), ignore_index=False) # Duplicate the records
            test_results_df = pd.concat([test_results_df, duplicated_records], ignore_index=False) # Append the duplicated records back to the original DataFrame (optional)
            
            # set values for output and n
            test_results_df['output'] = test_results_df['output'].astype(object) if 'output' in test_results_df.columns else None
            test_results_df.loc[index, 'output'] = outputs * len(records_to_duplicate)
            test_results_df['n'] = test_results_df['n'].astype(object) if 'n' in test_results_df.columns else None
            test_results_df.loc[index, 'n'] = ns * len(records_to_duplicate)
            
            test_results_df.to_csv(results_file_path, index=True, compression='gzip')
            
            time.sleep(wait_time_in_seconds)
            
            
def fail_safe_run(
    results_file_path,
    max_retries = 10,
    retries = 0,
):
    try:
        retries += 1    
        test_results_df = load_test_results(results_file_path)        
        run(test_results_df)
    except Exception as e:
        # if we get an error, wait for 60 seconds and try again
        if retries < max_retries:
            print(f"Error: {e}.\nWaiting for 60 seconds and trying again ({retries}).")
            time.sleep(60)
            fail_safe_run(results_file_path, retries=retries, max_retries=max_retries)
        else:
            print(f"Error: {e}.\nMax retries reached. Skipping.")

In [405]:
fail_safe_run(results_file_path)

Generating predictions for 152 records


Processing records:   0%|          | 0/152 [00:00<?, ?it/s]

In [336]:
test_results_df[test_results_df['output'].isna()].shape

(0, 13)

In [337]:
results_df = pd.read_csv(results_file_path, compression='gzip')
results_df.shape

(15200, 14)

#### Spec prompt

In [57]:
# MODEL_NAME = 'gpt-4-1106-preview'
MODEL_NAME = 'gpt-3.5-turbo-1106'

n = 100
id_labels = ['sample_id'] #['test_id', 'sample_id', 'sample_minor_id']
prompt_type = 'apispec' # 'examples' or 'apispec'
strategy = 'text2rep'
model_id = 'openai_' + MODEL_NAME.replace('-', '_')
examples_limit = 26
slug = strategy

test_df = load_eval_data()

results_file_path = f"./build/results/test-{str(test_df.shape[0])}-{model_id}-{slug}-n{n}-{prompt_type}-ex{examples_limit}.csv.gz"
if os.path.exists(results_file_path):
    raise ValueError(f"Results file already exists: {results_file_path}")

test_results_df = pd.read_csv(results_file_path, compression='gzip') if os.path.exists(results_file_path) else test_df.copy()
test_results_df.set_index(id_labels, inplace=True)
test_results_df.sort_index(inplace=True)

print(f"Saving results to {results_file_path}")

Saving results to ./build/results/test-152-openai_gpt_3.5_turbo_1106-text2rep-n100-apispec-ex26.csv.gz


In [58]:
prompt = build_prompt(
    prompt_type=prompt_type, 
    strategy=strategy, 
    input_data=examples_df.iloc[100],
    examples_df=examples_df,
    examples_limit=2
)
prompt

[{'role': 'system',
  'content': 'You are a skilled programmer. You will be provided with a text description and your task is to convert it into a dense variation of an Abstract Syntax Tree (AST) suitable for Python code.'},
 {'role': 'user',
  'content': 'Below are API specifications for implementing the corresponding dense AST for Python code.\n\n# MAP:\n\nclass Map(Action):\n    """\n    The Map class contains all the methods of a virtual assistant agent in the map domain.\n    """\n\n    @classmethod\n    def find_on_map(cls, location: Location) -> List[MapEntity]:\n        """\n        This class method finds places on the map.\n\n        Parameters\n        ----------\n        location : Location\n            The location to search for\n\n        Returns\n        -------\n        List[MapEntity]\n            A list of places in the form of map entities\n        """\n        pass\n\n# SHOPPING:\n\nclass Shopping(Action):\n    """\n    The Shopping class contains all the methods of

In [59]:
print("prompt", prompt)
prompt_str = "\n".join([p['content'] for p in prompt if p['role'] == 'user'])
print("promtp_str", prompt_str[:100])

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
base_prompt_tokens_len = len(tokenizer(prompt_str, max_length=51200, truncation=True)["input_ids"])
print(f"Base prompt tokens length: {base_prompt_tokens_len}")

prompt [{'role': 'system', 'content': 'You are a skilled programmer. You will be provided with a text description and your task is to convert it into a dense variation of an Abstract Syntax Tree (AST) suitable for Python code.'}, {'role': 'user', 'content': 'Below are API specifications for implementing the corresponding dense AST for Python code.\n\n# MAP:\n\nclass Map(Action):\n    """\n    The Map class contains all the methods of a virtual assistant agent in the map domain.\n    """\n\n    @classmethod\n    def find_on_map(cls, location: Location) -> List[MapEntity]:\n        """\n        This class method finds places on the map.\n\n        Parameters\n        ----------\n        location : Location\n            The location to search for\n\n        Returns\n        -------\n        List[MapEntity]\n            A list of places in the form of map entities\n        """\n        pass\n\n# SHOPPING:\n\nclass Shopping(Action):\n    """\n    The Shopping class contains all the methods 

In [60]:
openai_disabled = False
force = False
total_records = test_df.shape[0]
max_tokens = 512
wait_time_in_seconds = 20

# generate predictions
responses = []
print(f"Generating predictions for {total_records} records")
for i, row  in tqdm.notebook.tqdm(test_df.iterrows(), total=total_records, desc="Processing records"):   
    # check to see if we already have a result for this record
    index = tuple(row[id_label] for id_label in id_labels) if len(id_labels) > 1 else row[id_labels[0]]
    index = [index]
    if (not force) and ('output' in test_results_df.loc[index]) and (not any(pd.isnull(test_results_df.loc[index, 'output']))):
        # if we do, then skip this record
        time.sleep(0.1)
    elif not openai_disabled:
        # run the model, if we don't have a result
        messages = build_prompt(
            strategy=strategy, 
            prompt_type=prompt_type, 
            input_data=row, 
            examples_df=examples_df, 
            examples_limit=examples_limit
        )
        response = openai.ChatCompletion.create(
            model=MODEL_NAME, 
            messages=messages,
            max_tokens=max_tokens,
            n=n
        )
        responses.append(response)
        
        outputs = [x['message']['content'] for x in response['choices']]
        outputs = [output.replace("Code:\n", "").replace("```python", "").replace("```py", "").replace("```", "").strip() for output in outputs]
        ns = list(np.arange(n))
        
        # duplicate the records
        records_to_duplicate = test_results_df.loc[index] # Fetch the records
        duplicated_records = pd.concat([records_to_duplicate] * (n - 1), ignore_index=False) # Duplicate the records
        test_results_df = pd.concat([test_results_df, duplicated_records], ignore_index=False) # Append the duplicated records back to the original DataFrame (optional)
        
        # set values for output and n
        test_results_df['output'] = test_results_df['output'].astype(object) if 'output' in test_results_df.columns else None
        test_results_df.loc[index, 'output'] = outputs * len(records_to_duplicate)
        test_results_df['n'] = test_results_df['n'].astype(object) if 'n' in test_results_df.columns else None
        test_results_df.loc[index, 'n'] = ns * len(records_to_duplicate)
        
        test_results_df.to_csv(results_file_path, index=True, compression='gzip')
        
        time.sleep(wait_time_in_seconds)

Generating predictions for 152 records


Processing records:   0%|          | 0/152 [00:00<?, ?it/s]

In [36]:
results_file_path

'./build/results/test-152-openai_gpt_3.5_turbo_1106-rep2rep-n100-apispec-ex42-v2.csv.gz'

In [60]:
results_df = pd.read_csv(results_file_path, compression='gzip')

In [64]:
results_df.shape

(15200, 14)

In [63]:
def clean_output(output: str) -> str:
    if output is None:
        return None

    cleaned_output = output

    cleaned_output = re.sub(".*```(python|py)", "```", cleaned_output)
    x = cleaned_output.split("```")
    cleaned_output = x[1 if (len(x) > 1) else 0]

    regex_strings_to_remove = ['^Code:\n', '^Dense ast:\n', '^Dense AST:\n']
    for regex_string_to_remove in regex_strings_to_remove:
        cleaned_output = re.sub(regex_string_to_remove, "", cleaned_output)

    return cleaned_output


results_df['output'].apply(clean_output)


0        \t[ Module\n\t\t[ product_name = ProductName.r...
1        \n[ Module\n  [ If\n    [ test\n      [ Condit...
2        \t[ Module\n\t\t[ weather_attribute = WeatherA...
3        \t[ Module\n\t\t[ album = Album.resolve_from_t...
4        \t[ Module\n\t\t[ weather_attribute = WeatherA...
                               ...                        
15195    \t[ Module\n\t\t[ weather_attribute = WeatherA...
15196    \t[ Module\n\t\t[ date_time = DateTime.resolve...
15197    \t[ Module\n\t\t[ device_name = HomeDeviceName...
15198    \t[ Module\n\t\t[ weather_temperature = Weathe...
15199    \n[ Module\n  [ If\n    [ test\n      [ Weathe...
Name: output, Length: 15200, dtype: object

### gpt-3.5-turbo

#### Examples prompt

In [686]:
MODEL_NAME = 'gpt-3.5-turbo-1106'

In [687]:
n = 100
id_labels = ['sample_id'] #['test_id', 'sample_id', 'sample_minor_id']
prompt_type = 'examples' # 'examples' or 'apispec'
model_id = 'openai_' + MODEL_NAME.replace('-', '_')
slug = 'text2code'

In [688]:
test_df = load_eval_data()
print(test_df.shape)
test_df.head(3)

(152, 10)


,index,test_id,sample_id,sample_minor_id,text,code,test,imports,lang_rep,code_rep
0,0,0,0,NaN,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ Module [ product_name = ProductName.resolve_...
1,1,1_a,1,a,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ Module [ date_time = DateTime.resolve_from_t...
2,2,1_b,1,b,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ Module [ date_time = DateTime.resolve_from_t...


In [689]:
results_file_path = f"./build/results/test-{str(test_df.shape[0])}-{model_id}-{slug}-n{n}-{prompt_type}.csv.gz"
print(results_file_path)

./build/results/test-152-openai_gpt_3.5_turbo_1106-text2code-n100-examples.csv.gz


In [690]:
# loading the results file
test_results_df = pd.read_csv(results_file_path, compression='gzip') if os.path.exists(results_file_path) else test_df.copy()
test_results_df.set_index(id_labels, inplace=True)
test_results_df.sort_index(inplace=True)
test_results_df.head(3)

,index,test_id,sample_minor_id,text,code,test,imports,lang_rep,code_rep
sample_id,,,,,,,,,
0,0,0,NaN,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ Module [ product_name = ProductName.resolve_...
1,1,1_a,a,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ Module [ date_time = DateTime.resolve_from_t...
1,2,1_b,b,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ Module [ date_time = DateTime.resolve_from_t...


In [697]:
chat_format = True
# prompt = build_prompt(prompt_type=prompt_type, examples_limit=18, chat_format=chat_format)
prompt = build_prompt(prompt_type=prompt_type, examples_limit=90, chat_format=chat_format)
prompt_str = "\n".join([p['content'] for p in prompt if p['role'] == 'user'])
print(f"prompt_type: {prompt_type}\nprompt: {base_messages[:1000]}")

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
base_prompt_tokens_len = len(tokenizer(prompt_str, max_length=51200, truncation=True)["input_ids"])
print(f"Base prompt tokens length: {base_prompt_tokens_len}")

prompt_type: examples
prompt: [{'role': 'user', 'content': [{'role': 'user', 'content': 'Text: see if find my first reminders that I have a meeting at 3pm and there are and see if I got a reminder at mindnight in 2 days to bring the keys'}, {'role': 'assistant', 'content': 'Code:\nperson_reminded = Contact.resolve_from_text("my")\ncontent = Content.resolve_from_text("I have a meeting at 3pm")\nreminders = Reminders.find_reminders(person_reminded=person_reminded, content=content)\nreminders = first(reminders)\nResponder.respond(response=reminders)\ntest_reminders = bool(reminders)\nResponder.respond(response=test_reminders)\n\nperson_reminded = Contact.resolve_from_text("I")\ndate_time = DateTime.resolve_from_text("mindnight in 2 days")\ncontent = Content.resolve_from_text("bring the keys")\nreminders = Reminders.find_reminders(person_reminded=person_reminded, date_time=date_time, content=content)\ntest_reminders = bool(reminders)\nResponder.respond(response=test_reminders)'}, {'role': 

In [698]:
openai_disabled = False
force = False
total_records = test_df.shape[0]
max_tokens = 500
wait_time_in_seconds = 20

# generate predictions
responses = []
print(f"Generating predictions for {total_records} records")
for i, row  in tqdm.notebook.tqdm(test_df.iterrows(), total=total_records, desc="Processing records"):   
    # check to see if we already have a result for this record
    index = tuple(row[id_label] for id_label in id_labels) if len(id_labels) > 1 else row[id_labels[0]]
    index = [index]
    if (not force) and ('output' in test_results_df.loc[index]) and (not any(pd.isnull(test_results_df.loc[index, 'output']))):
        # if we do, then skip this record
        time.sleep(0.1)
    elif not openai_disabled:
        # run the model, if we don't have a result
        messages = [{
            "role": "system", 
            "content": "You are a skilled programmer. You will be provided with a text description and your task is to convert it into Python code. Below are examples of text descriptions and their corresponding Python code implementations."
            # "content": "You are a skilled programmer. You will be provided with a text description and your task is to convert it into Python code. Below is an API specifications for implementing the description in code."
        }] + prompt + [{
            "role": "user",
            "content": "Based on the previous examples, convert the following text into Python code:\n" + build_example_prompt(text=row["text"])
            # "content": "Based on the API spec, convert the following text into Python code:\n" + build_example_prompt(text=row["text"])
        }]
        response = openai.ChatCompletion.create(
            model=MODEL_NAME, 
            messages=messages,
            max_tokens=max_tokens,
            n=n
        )
        responses.append(response)
        
        outputs = [x['message']['content'] for x in response['choices']]
        outputs = [output.replace("Code:\n", "").replace("```python", "").replace("```py", "").replace("```", "").strip() for output in outputs]
        ns = list(np.arange(n))
        
        # duplicate the records
        records_to_duplicate = test_results_df.loc[index] # Fetch the records
        duplicated_records = pd.concat([records_to_duplicate] * (n - 1), ignore_index=False) # Duplicate the records
        test_results_df = pd.concat([test_results_df, duplicated_records], ignore_index=False) # Append the duplicated records back to the original DataFrame (optional)
        
        # set values for output and n
        test_results_df['output'] = test_results_df['output'].astype(object) if 'output' in test_results_df.columns else None
        test_results_df.loc[index, 'output'] = outputs * len(records_to_duplicate)
        test_results_df['n'] = test_results_df['n'].astype(object) if 'n' in test_results_df.columns else None
        test_results_df.loc[index, 'n'] = ns * len(records_to_duplicate)
        
        test_results_df.to_csv(results_file_path, index=True, compression='gzip')
        
        time.sleep(wait_time_in_seconds)

Generating predictions for 152 records


Processing records:   0%|          | 0/152 [00:00<?, ?it/s]

#### API specifications prompt

In [721]:
MODEL_NAME = 'gpt-3.5-turbo-1106'

n = 100
id_labels = ['sample_id'] #['test_id', 'sample_id', 'sample_minor_id']
prompt_type = 'apispec' # 'examples' or 'apispec'
model_id = 'openai_' + MODEL_NAME.replace('-', '_')
slug = 'text2code'
examples_limit = 2

test_df = load_eval_data()

results_file_path = f"./build/results/test-{str(test_df.shape[0])}-{model_id}-{slug}-n{n}-{prompt_type}-ex{examples_limit}.csv.gz"
test_results_df = pd.read_csv(results_file_path, compression='gzip') if os.path.exists(results_file_path) else test_df.copy()
test_results_df.set_index(id_labels, inplace=True)
test_results_df.sort_index(inplace=True)

In [722]:
chat_format = True
prompt = build_prompt(prompt_type=prompt_type, examples_limit=examples_limit, chat_format=chat_format)
print("prompt", prompt)
prompt_str = "\n".join([p['content'] for p in prompt if p['role'] == 'user'])
print("promtp_str", prompt_str[:100])

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
base_prompt_tokens_len = len(tokenizer(prompt_str, max_length=51200, truncation=True)["input_ids"])
print(f"Base prompt tokens length: {base_prompt_tokens_len}")

prompt [{'role': 'user', 'content': '\n    Convert the following task into Python code.\n    This is the API specification:\n# MAP:\n\nclass Map(Action):\n    """\n    The Map class contains all the methods of a virtual assistant agent in the map domain.\n    """\n\n    @classmethod\n    def find_on_map(cls, location: Location) -> List[MapEntity]:\n        """\n        This class method finds places on the map.\n\n        Parameters\n        ----------\n        location : Location\n            The location to search for\n\n        Returns\n        -------\n        List[MapEntity]\n            A list of places in the form of map entities\n        """\n        pass\n\n# SHOPPING:\n\nclass Shopping(Action):\n    """\n    The Shopping class contains all the methods of a virtual assistant agent in the shopping domain.\n    """\n\n    @classmethod\n    def find_products(\n        cls,\n        product_name: Optional[ProductName] = None,\n        product_attribute: Optional[ProductAttribute] 

In [725]:
openai_disabled = False
force = False
total_records = test_df.shape[0]
max_tokens = 500
wait_time_in_seconds = 20

# generate predictions
responses = []
print(f"Generating predictions for {total_records} records")
for i, row  in tqdm.notebook.tqdm(test_df.iterrows(), total=total_records, desc="Processing records"):   
    # check to see if we already have a result for this record
    index = tuple(row[id_label] for id_label in id_labels) if len(id_labels) > 1 else row[id_labels[0]]
    index = [index]
    if (not force) and ('output' in test_results_df.loc[index]) and (not any(pd.isnull(test_results_df.loc[index, 'output']))):
        # if we do, then skip this record
        time.sleep(0.1)
    elif not openai_disabled:
        # run the model, if we don't have a result
        messages = [{
            "role": "system", 
            # "content": "You are a skilled programmer. You will be provided with a text description and your task is to convert it into Python code. Below are examples of text descriptions and their corresponding Python code implementations."
            "content": "You are a skilled programmer. You will be provided with a text description and your task is to convert it into Python code. Below is an API specifications for implementing the description in code."
        }] + prompt + [{
            "role": "user",
            # "content": "Based on the previous examples, convert the following text into Python code:\n" + build_example_prompt(text=row["text"])
            "content": "Based on the API specifications and previous examples, convert the following text into Python code:\n" + build_example_prompt(text=row["text"])
        }]
        response = openai.ChatCompletion.create(
            model=MODEL_NAME, 
            messages=messages,
            max_tokens=max_tokens,
            n=n
        )
        responses.append(response)
        
        outputs = [x['message']['content'] for x in response['choices']]
        outputs = [output.replace("Code:\n", "").replace("```python", "").replace("```py", "").replace("```", "").strip() for output in outputs]
        ns = list(np.arange(n))
        
        # duplicate the records
        records_to_duplicate = test_results_df.loc[index] # Fetch the records
        duplicated_records = pd.concat([records_to_duplicate] * (n - 1), ignore_index=False) # Duplicate the records
        test_results_df = pd.concat([test_results_df, duplicated_records], ignore_index=False) # Append the duplicated records back to the original DataFrame (optional)
        
        # set values for output and n
        test_results_df['output'] = test_results_df['output'].astype(object) if 'output' in test_results_df.columns else None
        test_results_df.loc[index, 'output'] = outputs * len(records_to_duplicate)
        test_results_df['n'] = test_results_df['n'].astype(object) if 'n' in test_results_df.columns else None
        test_results_df.loc[index, 'n'] = ns * len(records_to_duplicate)
        
        test_results_df.to_csv(results_file_path, index=True, compression='gzip')
        
        time.sleep(wait_time_in_seconds)

Generating predictions for 152 records


Processing records:   0%|          | 0/152 [00:00<?, ?it/s]